In [2]:
x_int = 32768       # 16 bits
y_int = 54613       # 16 bits
z_int = 2505397     # 32 bits

In [3]:
# convert them into bits
# 16 bits for x and y
# 32 bits for z

x_bin = bin(x_int)
y_bin = bin(y_int)
z_bin = bin(z_int)
print("x in binary: ", x_bin)
print("y in binary: ", y_bin)
print("z in binary: ", z_bin)

x in binary:  0b1000000000000000
y in binary:  0b1101010101010101
z in binary:  0b1001100011101010110101


In [4]:
# concatenate x, y, and z into a single 64-bit binary number

encoded = x_bin[2:] + y_bin[2:] + z_bin[2:]
print("encoded: ", encoded)

encoded:  100000000000000011010101010101011001100011101010110101


In [8]:
# convert the 64-bit binary number back into the float64 number

decoded = int(encoded, 2)
print("decoded: ", decoded)
# show the dtype of decoded
print("dtype of decoded: ", type(decoded))

decoded:  9007428320770741
dtype of decoded:  <class 'int'>


In [6]:
# output the size of decoded in bytes

import sys
print("size of decoded in bytes: ", sys.getsizeof(decoded))

size of decoded in bytes:  32


In [13]:
def physical_space_to_integer_coordinates(x_mm, y_mm, z_mm):
    """
    Convert physical coordinates (x, y, z) in mm to integer representations 
    based on specified resolutions for 16-bit (x, y) and 32-bit (z) encoding.
    
    Parameters:
        x_mm (float): X coordinate in mm.
        y_mm (float): Y coordinate in mm.
        z_mm (float): Z coordinate in mm.
    
    Returns:
        tuple: (x_int, y_int, z_int) integer representations for encoding.
    """
    # Resolution values derived from bit allocations and physical space
    resolution_x = 600 / 65536  # 0.009155 mm per unit for x (16-bit)
    resolution_y = 600 / 65536  # 0.009155 mm per unit for y (16-bit)
    resolution_z = 1200 / 4294967296  # 0.0002794 mm per unit for z (32-bit)

    # Convert physical coordinates to integer representations
    x_int = int(x_mm / resolution_x)
    y_int = int(y_mm / resolution_y)
    z_int = int(z_mm / resolution_z)
    
    x_bin = bin(x_int)
    y_bin = bin(y_int)
    z_bin = bin(z_int)

    encoded = x_bin[2:] + y_bin[2:] + z_bin[2:]
    decoded = int(encoded, 2)
    return decoded

96384662287280330

In [12]:
def integer_coordinates_to_physical_space(decoded):

    # take the first 16 bits for x
    x_int = decoded >> 48
    # take the next 16 bits for y
    y_int = (decoded >> 32) & 0xFFFF
    # take the last 32 bits for z
    z_int = decoded & 0xFFFFFFFF

    # Resolution values derived from bit allocations and physical space
    resolution_x = 600 / 65536  # 0.009155 mm per unit for x (16-bit)
    resolution_y = 600 / 65536  # 0.009155 mm per unit for y (16-bit)
    resolution_z = 1200 / 4294967296  # 0.0002794 mm per unit for z (32-bit)

    # Convert integer representations to physical coordinates
    x_mm = x_int * resolution_x
    y_mm = y_int * resolution_y
    z_mm = z_int * resolution_z

    return x_mm, y_mm, z_mm

In [18]:
x_mm, y_mm, z_mm = 300, 400, 700
print("size of three coordinates in bytes: ", sys.getsizeof(x_mm) + sys.getsizeof(y_mm) + sys.getsizeof(z_mm))

encoded = physical_space_to_integer_coordinates(x_mm, y_mm, z_mm)
print("encoded: ", encoded)
print("size of encoded in bytes: ", sys.getsizeof(encoded))

x_mm_recon, y_mm_recon, z_mm_recon = integer_coordinates_to_physical_space(encoded)
print("x_mm_recon: ", x_mm_recon)
print("y_mm_recon: ", y_mm_recon)
print("z_mm_recon: ", z_mm_recon)
print("size of three coordinates in bytes: ", sys.getsizeof(x_mm_recon) + sys.getsizeof(y_mm_recon) + sys.getsizeof(z_mm_recon))

size of three coordinates in bytes:  84
encoded:  9223559686481335637
size of encoded in bytes:  36
x_mm_recon:  300.0
y_mm_recon:  399.993896484375
z_mm_recon:  699.9999999068677
size of three coordinates in bytes:  72


In [22]:
x_space_mm = 600
y_space_mm = 600
z_space_mm = 400
# we can distribute different bits to x, y, and z for different resolution requirements

x_bits = 24
y_bits = 24
z_bits = 16

def physical_space_to_integer_coordinates_v2(x_mm, y_mm, z_mm):

    # Resolution values derived from bit allocations and physical space
    resolution_x = x_space_mm / (2 ** x_bits)
    resolution_y = y_space_mm / (2 ** y_bits)
    resolution_z = z_space_mm / (2 ** z_bits)

    # Convert physical coordinates to integer representations
    x_int = int(x_mm / resolution_x)
    y_int = int(y_mm / resolution_y)
    z_int = int(z_mm / resolution_z)
    
    x_bin = bin(x_int)
    y_bin = bin(y_int)
    z_bin = bin(z_int)

    encoded = x_bin[2:] + y_bin[2:] + z_bin[2:]
    encoded = int(encoded, 2)
    return encoded

In [23]:
def integer_coordinates_to_physical_space_v2(encoded, x_bits=24, y_bits=24, z_bits=16):

    # take the first x_bits bits for x
    x_int = encoded >> (y_bits + z_bits)
    # take the next y_bit bits for y
    y_int = (encoded >> z_bits) & (2**y_bits - 1)
    # take the last z_bit bits for z
    z_int = encoded & (2**z_bits - 1)

    # Resolution values derived from bit allocations and physical space
    resolution_x = x_space_mm / (2 ** x_bits)
    resolution_y = y_space_mm / (2 ** y_bits)
    resolution_z = z_space_mm / (2 ** z_bits)

    # Convert integer representations to physical coordinates
    x_mm = x_int * resolution_x
    y_mm = y_int * resolution_y
    z_mm = z_int * resolution_z

    return x_mm, y_mm, z_mm

In [26]:
# example we have the location (300, 400, 200) in mm
x_mm, y_mm, z_mm = 300, 400, 200

assert x_mm < x_space_mm
assert y_mm < y_space_mm
assert z_mm < z_space_mm

encoded_coor = physical_space_to_integer_coordinates_v2(x_mm, y_mm, z_mm)
print("encoded_coor: ", encoded_coor)

recon_coors = integer_coordinates_to_physical_space_v2(encoded_coor)
print("recon_coors: ", recon_coors)


encoded_coor:  9223372769862516736
recon_coors:  (300.0, 399.9999761581421, 200.0)
